In [0]:
import pandas as pd
import numpy as np
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader

In [42]:
from google.colab import files
uploaded = files.upload()

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Line.txt to Line.txt


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [49]:
sp_text = ""

chars = []
i = 0
for char in open('character.txt'):
    chars.append(char.strip() + ":\t")
    if i>=2000:
        break
    i = i+1
lines = [] 

i = 0
for line in open('Line.txt', encoding='ISO-8859-1' ):
        lines.append(line)
        if i>=2000:
            break
        i = i+1
        
for i in range(len(lines)):
    total = chars[i] + lines[i]
    sp_text += total
    
vocab = list(set(sp_text))
vocab_stoi = {s: i for i, s in enumerate(vocab)}
vocab_itos = {i: s for i, s in enumerate(vocab)}
print(len(sp_text))
print(vocab_stoi)

146675
{'h': 0, 'p': 1, '"': 2, 'Z': 3, '7': 4, '/': 5, '3': 6, '-': 7, 'Y': 8, '©': 9, 'n': 10, 'W': 11, '.': 12, '&': 13, 'B': 14, 'S': 15, '\n': 16, 'z': 17, '9': 18, 'd': 19, 'I': 20, '8': 21, 'r': 22, '±': 23, 'C': 24, ';': 25, 's': 26, 'A': 27, 'f': 28, 'Á': 29, '5': 30, '2': 31, '(': 32, 'q': 33, 'G': 34, 'N': 35, "'": 36, 'g': 37, 't': 38, 'P': 39, 'a': 40, 'j': 41, 'V': 42, '4': 43, '6': 44, '¬': 45, 'F': 46, 'D': 47, 'Ì': 48, 'e': 49, 'H': 50, 'T': 51, '1': 52, 'å': 53, 'i': 54, ')': 55, '0': 56, 'b': 57, 'O': 58, 'c': 59, 'Q': 60, 'L': 61, 'm': 62, ' ': 63, 'k': 64, '+': 65, 'K': 66, '%': 67, 'v': 68, '\t': 69, 'l': 70, 'M': 71, 'y': 72, '\ufeff': 73, 'u': 74, 'w': 75, 'À': 76, 'U': 77, 'X': 78, '!': 79, ',': 80, 'E': 81, 'J': 82, 'x': 83, ':': 84, 'o': 85, 'R': 86, '?': 87}


In [7]:
import random
random.seed(10)

sp_len = len(sp_text) #get the length of the south park text 

def random_chunk(chunk_len=1000):
    """Return a random subsequence from `spam_text`"""
    start_index = random.randint(0, sp_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return sp_text[start_index:end_index] #gets a random chunk of size chunk len

def text_to_numpy(text, vocab=vocab):  #
    """Return a tensor containing the indices of characters in `text`."""
    indices = [vocab_stoi[ch] for ch in text]
    return np.asarray(indices)

  
example = text_to_numpy(random_chunk())
# print(example)


[49 40  0 ... 70 58 65]


In [21]:
## based on ideas from https://github.com/thomasxmeng/Project-tv-script-generation-RNN/blob/master/dlnd_tv_script_generation_7.ipynb

def make_data_loader(sp_txt_np, inp_len, batch_size): 
    #create input target sequence pairs 
    length = len(sp_txt_np) #figure out how long the sp_text_np array is
    
    inp_list = []
    target_list = []
    start_ind = 0
    end_ind = inp_len
    tar_ind = inp_len
    
    #we want to return inputs and targets, where inputs are a vector array of sequence length 
    while (length-start_ind)> inp_len: #while we can still create a batch target pair 
      inp_list.append(sp_txt_np[start_ind:end_ind])
      target_list.append(sp_txt_np[tar_ind])
      
      #shift everything by 1
      start_ind +=1
      end_ind +=1
      tar_ind +=1
      
    #convert the inputs and targets into np arrays in order to convert them to torch tensors
    inputs = np.asarray(inp_list)
    targets = np.asarray(target_list)
      
    #create a data loader 
    data = TensorDataset(torch.from_numpy(inputs), torch.from_numpy(targets))
    data_loader = torch.utils.data.DataLoader(data, shuffle=True, batch_size=batch_size) #shuffle and add batch size
   
  # return a dataloader
    return data_loader
  
  
loader = make_data_loader(text_to_numpy(sp_text),30,1)
print(text_to_numpy(sp_text)[:4])
for i,j in loader:
    print(i,j)
    print(i.shape,len(j))
    break

[67 32 30 38]
tensor([[56,  0,  4, 66, 50, 53, 14, 62, 62, 45,  6, 19, 60,  4, 36, 27, 26, 26,
         27, 35, 46, 60, 54,  4, 50, 53, 61, 51, 38,  5]]) tensor([27])
torch.Size([1, 30]) 1


In [0]:
class Cartmanboi(nn.Module):
    def __init__(self, vocab_size, hidden_size, n_layers=1,dropout=0.2):
        super(Cartmanboi, self).__init__()
        # RNN attributes
        self.name = "c1"
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        # identity matrix for generating one-hot vectors on the chars 
        self.ident = torch.eye(vocab_size)
        # recurrent neural network
        self.rnn = nn.GRU(vocab_size, hidden_size, n_layers, batch_first=True,dropout=dropout)
        # a fully-connect layer that decodes the RNN output to
        # a distribution over the vocabulary
        self.decoder = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, inp, hidden):
        batch_size = inp.size(0) #figure out what the size of the input is 
        # generate one-hot vectors from token indices
        inp = self.ident[inp]
        # obtain the next output and hidden state
        #print(inp.shape)
        output, hidden = self.rnn(inp, hidden)
        output = self.decoder(output.squeeze(0))
        output = output.view(batch_size, -1, self.vocab_size)

        output = output[:,-1] #take the last output 
        return output, hidden

    def init_hidden(self,batch_size): #this returns a hidden size the same size as the batch size 
        return torch.zeros(self.n_layers, batch_size, self.hidden_size)

In [0]:
current_seq = np.full((1, 30), 30)
print(current_seq.shape)
prime_str='Kyle:'
prime_input = text_to_numpy(prime_str)
current_seq[-1][len(current_seq)-len(prime_input)-1:] = prime_input
print(current_seq)
#print(current_seq[-1][-1])
print(prime_input)

(1, 30)
[[30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
  30 66 60 54 24  1]]
[66 60 54 24  1]


In [0]:
def get_model_name(name, batch_size, learning_rate, epoch):
    """ Generate a name for the model consisting of all the hyperparameter values

    Args:
        config: Configuration object containing the hyperparameters
    Returns:
        path: A string with the hyperparameter name and value concatenated
    """
    model_name = 'model_{0}_bs{1}_lr{2}_epoch{3}'.format(name,
                                                   batch_size,
                                                   learning_rate,
                                                   epoch)
    path = F"users/litongzheng/desktop/paths/{model_name}"
    return path

In [0]:
def evaluate(model, prime_str='Kyle:', sq_length=30, predict_len=500, temperature=0.8):
    hidden = model.init_hidden(1) #create a hidden layer the size of our prime string 
    prime_input = text_to_numpy(prime_str) #convert out prime string to a np array vector 
    
    
    #create a matrix the same size as our input batches 
    current_seq = np.full((1, sq_length), 63) #create a vector to hold the predicted characters 
    
    
    current_seq[-1][len(current_seq)-len(prime_input)-1:] = prime_input
    predicted = prime_str #predicted holds the predicted sequence 
    current_seq = torch.from_numpy(current_seq)
    
    for p in range(predict_len):  #for however many characters we want to predict 
        current_seq = torch.LongTensor(current_seq) #turn our current sequence into a long tensor 
        output, hidden = model(current_seq, hidden)
#         print(hidden.shape)
#         print(current_seq.shape)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = int(torch.multinomial(output_dist, 1)[0])
        # Add predicted character to string and use as next input
        predicted_char = vocab_itos[top_i] #figure out what the predicted character is
        predicted += predicted_char
        
#         print(current_seq)
        current_seq = np.roll(current_seq, -1, 1)
#         print(current_seq)
        current_seq[-1][-1] = top_i
#         print(current_seq)

    return predicted

def train(model, num_epochs=50, batch_size = 128, lr=0.001):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    #convert the text into a vector aray, how many previous chars to use for prediction, the size of the batch
    train_loader =  make_data_loader(text_to_numpy(sp_text),30,batch_size) 

    for epoch in range(num_epochs):
        for inp,tar in train_loader: #note that we don't have a validation loader 
            if len(tar) == batch_size:
                optimizer.zero_grad()
                hidden = model.init_hidden(batch_size)
                output,_ = model(inp,hidden)
                loss = criterion(output, tar)
                loss.backward()
                optimizer.step() 
        print("[Epoch %d] Loss %f" % (epoch+1, float(loss)))
#         model_path = get_model_name(model.name, batch_size, lr, epoch)
#         torch.save(model.state_dict(), model_path)
        print(evaluate(model,prime_str="Kyle:"))



In [51]:
model = Cartmanboi(len(vocab),128,2)
train(model,batch_size=128)

[Epoch 1] Loss 2.207885
Kyle:	The you nes. I'w su bid and can sus is wat realles ston'm se mpadthe chan's us you Favend vehinny Goy seass all the anders finy! 
Ally:	Whan!  
OOs Cartman:	Wh, wkad nouss. Gorad-goe dress the of got in norest.  
Chaf:	Wery shoik boing ef chod youn's wus of bave at one to noucloon wo preat! 
Ceral:	Staed to gock on moalayn't ara got a sake taell, shole stom we so doot tomed Famigh. 
Chet:	ManUy reat Jettall chas you leang wory wo hyod, the rerid con cand you  piot becan's thas. Wens sowr the al
[Epoch 2] Loss 1.812071
Kyle:	In'  
Cartor:	Ando, tole, as comeatyoo??. Cartman! Nos, now.  All the cart more bodt in the wertie.  Canno, quohe to as stop goons timeveryos the fare houstment with in they stay migete do ming the mortalist of erident herion and houth 3 hardy for Jasnes.  
Kyle:	Who presionny to winnd the knowel you come oof the proald diantare to gate istert of eresess pregided? 
Kyle:	You stell got on thour! 
Cartman:	Oh in!.  Maykien Clude, is time?

KeyboardInterrupt: ignored

In [57]:
print(evaluate(model ,prime_str = "Cartman:"))

Cartman: Warvis, but, I want what's me out of the starting to wine the book me and the Addon't yeald is the From you. 
Liane:	Oh he's doing right.  Maybe hey listen shouldn't comar proghed stop of the earther to come on in a so mance I'm come foundrevers aren't again. 
Kyle:	Oh my Goddamned, becomales hone. I'm all right one.  
Stan:	What?! 
Gary:	Oh... 
Randy:	What that's from Cloodey?  he's not a joke!  
Stan:	Yeah, but the man we had pulled at personithing here. 
Cartman:	Yeah, we days it one, buttbo
